In [16]:
import torch
from torchvision import models, transforms
from PIL import Image
import cv2
import os
from tqdm import tqdm 
import pandas as pd

/tmp/ipykernel_447/4204378327.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [11]:
#pip install torch torchvision Pillow opencv-python

In [10]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [6]:

def preprocess_image(frame):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(frame).unsqueeze(0)

def segment_image(img_tensor):
    model = models.segmentation.deeplabv3_resnet101(pretrained=True)
    model.eval()
    with torch.no_grad():
        output = model(img_tensor)['out'][0]
    output_predictions = output.argmax(0)
    return output_predictions

def calculate_foreground_background(segmented_img):
    foreground = (segmented_img != 0).sum()
    total_pixels = segmented_img.numel()
    background = total_pixels - foreground
    foreground_percentage = (foreground.item() / total_pixels) * 100
    background_percentage = (background.item() / total_pixels) * 100
    return foreground.item(), background_percentage

def classify_frames(video_path):
    vidcap = cv2.VideoCapture(video_path)
    if not vidcap.isOpened():
        print(f"Failed to open video file: {video_path}")
        return None

    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1  # Adjusting to avoid reading beyond the last frame

    if total_frames == 0:
        print("The video has no frames.")
        return None

    frame_indices = [int(total_frames * i / 4) for i in range(5)]
    frame_indices[-1] = min(frame_indices[-1], total_frames - 1)

    foreground_pixels_list = []
    background_percentage_list = []

    for index in frame_indices:
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, index)
        success, frame = vidcap.read()

        if not success:
            print(f"Could not read the frame at index {index}")
            continue

        img_tensor = preprocess_image(frame)
        segmented_img = segment_image(img_tensor)
        foreground_pixels, background_percentage = calculate_foreground_background(segmented_img)

        foreground_pixels_list.append(foreground_pixels)
        background_percentage_list.append(background_percentage)

    vidcap.release()

    # Calculate and return the average of foreground pixels and background percentage
    avg_foreground_pixels = sum(foreground_pixels_list) / len(foreground_pixels_list)
    avg_background_percentage = sum(background_percentage_list) / len(background_percentage_list)

    return avg_foreground_pixels, avg_background_percentage

In [17]:
def classify_folder_scenes(folder_path):
    video_files = [f for f in os.listdir(folder_path) 
                   if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.mp4')]
    data = []

    # Wrap video_files with tqdm for progress tracking
    for video_file in tqdm(video_files, desc='Classifying videos'):
        video_id = video_file[:4]  # Extract video ID from filename
        video_path = os.path.join(folder_path, video_file)
        avg_foreground_pixels, avg_background_percentage = classify_frames(video_path)
        data.append([video_id, avg_foreground_pixels, avg_background_percentage])

    df = pd.DataFrame(data, columns=['Video ID', 'Average Foreground Pixels', 'Average Background Percentage'])
    return df

In [18]:
df_scenes = classify_folder_scenes('AlgonautsVideos268_All_30fpsmax')

Classifying videos:   0%|          | 0/1102 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Classifying videos: 100%|██████████| 1102/1102 [1:26:48<00:00,  4.73s/it]


In [19]:
df_scenes

,Video ID,Average Foreground Pixels,Average Background Percentage
0,1012,20614.2,58.916215
1,0284,0.0,100.000000
2,0039,38298.8,23.671078
3,0853,4378.8,91.273119
4,1035,15217.4,69.671955
...,...,...,...
1097,0445,1558.2,96.894531
1098,0542,10990.0,78.097098
1099,0565,25255.2,49.666773
1100,1003,0.0,100.000000


In [20]:
# Assuming df_scenes is your DataFrame variable name
df_scenes.to_csv('df_scenes.csv', index=False)